# Configuration Best Practices

# General Configuration Tips

# "Naked" Pods versus ReplicaSets, Deployments, and Jobs

# Services

# Using Labels

# Using kubectl 

# Environment Variables

### 方法一: 直接使用指令

In [ ]:
kubectl run web --image nginx \
  --env='USERNAME=alice' \
  --env 'PASSWORD=123456' \
  -l 'app=web,env=test' \
  -o yaml --dry-run=client \
  > test.yaml

In [ ]:
kubctl apply -f test.yaml

In [ ]:
$env

### 方法二: 使用 yaml 文件

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: pod-env
spec:
  containers:
  - name: producer
    image: busybox
    command: ["sh", "-c", "while true; do echo $NAME >> /tmp/index.html; sleep 10; done"]
    env:
    - name: NAMEㄔㄛ
      value: Hello World

In [ ]:
kubectl apply -f pod-env.yml
kubectl get pods
kubectl exec pod-env -- more /tmp/index.html

##### 範例

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: mysql
spec:
  containers:
  - name: mysql
    image: mysql:8.0
    env:
    - name: MYSQL_ROOT_PASSWORD
      value: root

In [ ]:
使用 yaml 檔建立 pod

In [ ]:
kubectl apply -f mysql.yml

In [ ]:
檢查環境變數有沒有設定成功

In [ ]:
$env

In [ ]:
mysql -h 127.0.0.1 -u root -p

### 方法三: 使用 ConfigMap

##### 使用指令

In [ ]:
kubectl create configmap mysql-cfg \
  --from-literal=MYSQL_ROOT_PASSWORD=root \
  --from-literal=MYSQL_USER=demo \
  --from-literal=MYSQL_PASSWORD=demo

In [ ]:
kubectl get configMap

##### 使用檔案

In [ ]:
bind 127.0.0.1
port 6379
maxclients 10000
maxmemory 50mb
maxmemory-policy volatile-lru
syslog-enabled yes
dir /var/lib/redis
dbfilename redis.dump.rdb
databases 1
appendfsync everysec
save 600 10

In [ ]:
kubectl create configmap redis-config --from-file=my-redis.conf

In [ ]:
kubectl get confexitigMap

In [ ]:
kubectl describe configMap

##### 使用 yaml 檔

In [ ]:
apiVersion: v1
kind: ConfigMap
metadata:
  name: appconfig
data:
  MYSQL_ROOT_PASSWORD: root
  MYSQL_USER: demo
  MYSQL_PASSWORD: demo

In [ ]:
kubectl apply -f appconfig.yaml

In [ ]:
kubectl get configmaps

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: mysql
spec:
  containers:
  - name: mysql
    image: mysql:8.0
    envFrom:
      - configMapRef:
          name: appconfig

##### 使用 Volume 的方式

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: pod-env
spec:
  volumes:
  - name: appconfig
    configMap:
      name: appconfig
  containers:
  - name: busybox
    image: busybox
    command: ["sh", "-c", "while true; do echo $(date) >> /tmp/index.html; sleep 10; done"]
    volumeMounts:
    - name: appconfig
      mountPath: "/etc/appconfig"

### 方法四: 使用 Secrets

##### Uses for Secrets

##### Working with Secrets

##### 方法一: create Secret using kubectl command

##### Create a Secret

##### Use raw data

In [ ]:
kubectl create secret generic db-user-pass \
    --from-literal=username=devuser \
    --from-literal=password='S!B\*d$zDsb='

##### Use source files

In [ ]:
echo -n 'admin' | base64 > ./username.txt
echo -n 'S!B\*d$zDsb=' | base64 > ./password.txt

In [ ]:
kubectl create secret generic db-user-pass \
    --from-file=./username.txt \
    --from-file=./password.txt

##### Verify the Secret

In [ ]:
kubectl get secrets

In [ ]:
kubectl describe secret db-user-pass

##### Decode the Secret

In [ ]:
kubectl get secret db-user-pass -o jsonpath='{.data}'

In [ ]:
echo 'UyFCXCpkJHpEc2I9' | base64 --decode

In [ ]:
kubectl get secret db-user-pass -o jsonpath='{.data.password}' | base64 --decode

##### Edit a Secret

In [ ]:
kubectl edit secrets <secret-name>

In [ ]:
# Please edit the object below. Lines beginning with a '#' will be ignored,
# and an empty file will abort the edit. If an error occurs while saving this file, it will be
# reopened with the relevant failures.
#
apiVersion: v1
data:
  password: UyFCXCpkJHpEc2I9
  username: YWRtaW4=
kind: Secret
metadata:
  creationTimestamp: "2022-06-28T17:44:13Z"
  name: db-user-pass
  namespace: default
  resourceVersion: "12708504"
  uid: 91becd59-78fa-4c85-823f-6d44436242ac
type: Opaque

##### Clean up

In [ ]:
kubectl delete secret db-user-pass

##### 方法二: create Secret from config file

##### Create the Secret 

In [ ]:
echo -n 'admin' | base64
echo -n '1f2d1e2e67df' | base64

In [ ]:
YWRtaW4=
MWYyZDFlMmU2N2Rm

In [ ]:
apiVersion: v1
kind: Secret
metadata:
  name: mysecret
type: Opaque
data:
  username: YWRtaW4=
  password: MWYyZDFlMmU2N2Rm

In [ ]:
kubectl apply -f ./secret.yaml

In [ ]:
secret/mysecret created

##### Specify unencoded data when creating a Secret

In [ ]:
apiUrl: "https://my.api.com/api/v1"
username: "<user>"
password: "<password>"

In [ ]:
apiVersion: v1
kind: Secret
metadata:
  name: mysecret
type: Opaque
stringData:
  config.yaml: |
    apiUrl: "https://my.api.com/api/v1"
    username: <user>
    password: <password>    

In [ ]:
apiVersion: v1
data:
  config.yaml: YXBpVXJsOiAiaHR0cHM6Ly9teS5hcGkuY29tL2FwaS92MSIKdXNlcm5hbWU6IHt7dXNlcm5hbWV9fQpwYXNzd29yZDoge3twYXNzd29yZH19
kind: Secret
metadata:
  creationTimestamp: 2018-11-15T20:40:59Z
  name: mysecret
  namespace: default
  resourceVersion: "7225"
  uid: c280ad2e-e916-11e8-98f2-025000000001
type: Opaque

##### Specify both data and stringData

In [ ]:
apiVersion: v1
kind: Secret
metadata:
  name: mysecret
type: Opaque
data:
  username: YWRtaW4=
stringData:
  username: administrator

In [ ]:
apiVersion: v1
data:
  username: YWRtaW5pc3RyYXRvcg==
kind: Secret
metadata:
  creationTimestamp: 2018-11-15T20:46:46Z
  name: mysecret
  namespace: default
  resourceVersion: "7579"
  uid: 91460ecb-e917-11e8-98f2-025000000001
type: Opaque

##### Edit a Secret 

In [ ]:
echo -n 'birdsarentreal' | base64

In [ ]:
YmlyZHNhcmVudHJlYWw=

In [ ]:
apiVersion: v1
kind: Secret
metadata:
  name: mysecret
type: Opaque
data:
  username: YWRtaW4=
  password: YmlyZHNhcmVudHJlYWw=

In [ ]:
kubectl apply -f ./secret.yaml

##### Clean up

In [ ]:
kubectl delete secret mysecret

### Constraints on Secret names and data

### Size limit

### Editing a Secret

In [ ]:
kubectl edit secrets mysecret

In [ ]:
# Please edit the object below. Lines beginning with a '#' will be ignored,
# and an empty file will abort the edit. If an error occurs while saving this file, it will be
# reopened with the relevant failures.
#
apiVersion: v1
data:
  username: YWRtaW4=
  password: MWYyZDFlMmU2N2Rm
kind: Secret
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: { ... }
  creationTimestamp: 2020-01-22T18:41:56Z
  name: mysecret
  namespace: default
  resourceVersion: "164619"
  uid: cfee02d6-c137-11e5-8d73-42010af00002
type: Opaque

### Using a Secret

### Optional Secrets

### Using Secrets as files from a Pod

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: mypod
spec:
  containers:
  - name: mypod
    image: redis
    volumeMounts:
    - name: foo
      mountPath: "/etc/foo"
      readOnly: true
  volumes:
  - name: foo
    secret:
      secretName: mysecret
      optional: false # default setting; "mysecret" must exist

### Projection of Secret keys to specific paths

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: mypod
spec:
  containers:
  - name: mypod
    image: redis
    volumeMounts:
    - name: foo
      mountPath: "/etc/foo"
      readOnly: true
  volumes:
  - name: foo
    secret:
      secretName: mysecret
      items:
      - key: username
        path: my-group/my-username

### Consuming Secret values from volumes

In [ ]:
ls /etc/foo/

In [ ]:
cat /etc/foo/username

In [ ]:
cat /etc/foo/password

### Mounted Secrets are updated automatically

### Using Secrets as environment variables

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: secret-env-pod
spec:
  containers:
  - name: mycontainer
    image: redis
    env:
      - name: SECRET_USERNAME
        valueFrom:
          secretKeyRef:
            name: mysecret
            key: username
            optional: false # same as default; "mysecret" must exist
                            # and include a key named "username"
      - name: SECRET_PASSWORD
        valueFrom:
          secretKeyRef:
            name: mysecret
            key: password
            optional: false # same as default; "mysecret" must exist
                            # and include a key named "password"
  restartPolicy: Never

### Invalid environment variables

In [ ]:
kubectl get events

### Consuming Secret values from environment variables 

In [ ]:
echo "$SECRET_USERNAME"

In [ ]:
admin

In [ ]:
echo "$SECRET_PASSWORD"

In [ ]:
1f2d1e2e67df

### 範例: As container environment variables 

In [ ]:
apiVersion: v1
kind: Secret
metadata:
  name: mysecret
type: Opaque
data:
  USER_NAME: YWRtaW4=
  PASSWORD: MWYyZDFlMmU2N2Rm

In [ ]:
Create the Secret:

In [ ]:
kubectl apply -f mysecret.yaml

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: secret-test-pod
spec:
  containers:
    - name: test-container
      image: registry.k8s.io/busybox
      command: [ "/bin/sh", "-c", "env" ]
      envFrom:
      - secretRef:
          name: mysecret
  restartPolicy: Never

In [ ]:
kubectl logs secret-test-pod

### 範例: Pod with SSH keys

In [ ]:
kubectl create secret generic ssh-key-secret --from-file=ssh-privatekey=/path/to/.ssh/id_rsa --from-file=ssh-publickey=/path/to/.ssh/id_rsa.pub

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: secret-test-pod
  labels:
    name: secret-test
spec:
  volumes:
  - name: secret-volume
    secret:
      secretName: ssh-key-secret
  containers:
  - name: ssh-test-container
    image: mySshImage
    volumeMounts:
    - name: secret-volume
      readOnly: true
      mountPath: "/etc/secret-volume"

### 範例:  Define a Command and Arguments for a Container

##### Define a command and arguments when you create a Pod

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: command-demo
  labels:
    purpose: demonstrate-command
spec:
  containers:
  - name: command-demo-container
    image: debian
    command: ["printenv"]
    args: ["HOSTNAME", "KUBERNETES_PORT"]
  restartPolicy: OnFailure

In [ ]:
kubectl apply -f https://k8s.io/examples/pods/commands.yaml

In [ ]:
kubectl get pods

In [ ]:
kubectl logs command-demo

##### Use environment variables to define arguments 

In [ ]:
env:
- name: MESSAGE
  value: "hello world"
command: ["/bin/echo"]
args: ["$(MESSAGE)"]

##### Run a command in a shell

In [ ]:
command: ["/bin/sh"]
args: ["-c", "while true; do echo hello; sleep 10;done"]